In [14]:
import cv2
import csv
import os
import pandas as pd
from datetime import datetime, timedelta
from PIL import Image
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [15]:
def match_images_with_csv(csv_file_path, images_folder_path, timezone_offset=7):
    timestamp_pm10_mapping = {}
    with open(csv_file_path, mode='r') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            now_timestamp = datetime.fromisoformat(row['Now Timestamp'])
            now_timestamp += timedelta(hours=timezone_offset)
            formatted_timestamp = now_timestamp.strftime('%Y%m%d_%H%M')
            formatted_timestamp += "00"
            timestamp_pm10_mapping[formatted_timestamp] = row['AQI']

    x = []
    y = []

    for image_file in os.listdir(images_folder_path):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(images_folder_path, image_file)
            image_timestamp = image_file.split('.')[0]

            # Match the image timestamp with the CSV data
            if image_timestamp in timestamp_pm10_mapping:
                pm10_value = timestamp_pm10_mapping[image_timestamp]
                pm10_class = check_value(int(pm10_value))
                y.append(pm10_class)
                x.append(image_path)

    return x, y

def check_value(value):
    if 0 <= value < 50:
        return 0
    elif 50 <= value < 150:
        return 1
    elif 150 <= value < 250:
        return 2
    elif 250 <= value < 350:
        return 3
    elif 350 <= value < 450:
        return 4
    elif 450 <= value <= 600:
        return 5
    else:
        return "Out of range"

In [16]:
import os
os.chdir('/home/ubuntu/project/predict-air-quality-with-ANN')
x, y = match_images_with_csv('aqi-dataset/air_quality_data.csv', 'image-dataset/')

In [17]:
df = pd.DataFrame({"image_path": x, "pm10_value": y})

In [18]:
filenames = df['image_path'].values
labels = df['pm10_value'].values

# Create a TensorFlow dataset from the dataframe
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# Load and preprocess images
def parse_image(filename, label):
    file_content = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(file_content, channels=3)
    image = tf.image.resize(image, [227,227])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply the load_and_preprocess_image function to each element in the dataset
dataset = dataset.map(parse_image)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
train_dataset = train_dataset.batch(batch_size=2,drop_remainder=True)

val_dataset = dataset.skip(train_size)
val_dataset = val_dataset.batch(batch_size=2,drop_remainder=True)

for image,label in dataset.take(100):
  print(label)

2024-06-04 02:15:22.195040: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-06-04 02:15:22.295648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [146]
	 [[{{node Placeholder/_1}}]]


tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, BatchNormalization)

def alexnet_model(input_shape=(227, 227, 3)):
    model = Sequential([
        # 1st Convolutional Layer
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 2nd Convolutional Layer
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 3rd, 4th, and 5th Convolutional Layers
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # Flattening Layer
        Flatten(),

        # 1st Dense Layer
        Dense(4096, activation='relu'),
        Dropout(0.5),

        # 2nd Dense Layer
        Dense(4096, activation='relu'),
        Dropout(0.5),

        # 3rd Dense Layer
        Dense(1000, activation='relu'),  # 1000 refers to the number of classes in the original AlexNet paper
        Dropout(0.5),

        # Output Layer
        Dense(10, activation='softmax')  # Adjust the number of units to match the number of classes you have
    ])

    return model

# Create the model
model = alexnet_model()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [20]:
epochs = 1000

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

Epoch 1/1000
58/58 [==============================] - 20s 343ms/step - loss: 0.7914 - accuracy: 0.4741 - val_loss: 0.7187 - val_accuracy: 0.5333
Epoch 644/1000
58/58 [==============================] - 19s 325ms/step - loss: 0.7937 - accuracy: 0.5517 - val_loss: 0.7182 - val_accuracy: 0.5333
Epoch 645/1000
58/58 [==============================] - 19s 319ms/step - loss: 0.8087 - accuracy: 0.4569 - val_loss: 0.7190 - val_accuracy: 0.5333
Epoch 646/1000
58/58 [==============================] - 19s 323ms/step - loss: 0.7973 - accuracy: 0.4741 - val_loss: 0.7185 - val_accuracy: 0.5333
Epoch 647/1000
58/58 [==============================] - 19s 324ms/step - loss: 0.7959 - accuracy: 0.4138 - val_loss: 0.7179 - val_accuracy: 0.5333
Epoch 648/1000
58/58 [==============================] - 19s 320ms/step - loss: 0.8079 - accuracy: 0.4483 - val_loss: 0.7185 - val_accuracy: 0.5333
Epoch 649/1000
58/58 [==============================] - 19s 325ms/step - loss: 0.8032 - accuracy: 0.5259 - val_loss: 0.7

2024-06-04 02:15:23.026507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [146]
	 [[{{node Placeholder/_1}}]]


58/58 [==============================] - ETA: 0s - loss: 4.6635 - accuracy: 0.4310

2024-06-04 02:15:43.073012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [146]
	 [[{{node Placeholder/_1}}]]


58/58 [==============================] - 21s 318ms/step - loss: 4.6635 - accuracy: 0.4310 - val_loss: 0.8978 - val_accuracy: 0.5333
Epoch 2/1000
58/58 [==============================] - 18s 317ms/step - loss: 0.9809 - accuracy: 0.4397 - val_loss: 0.7278 - val_accuracy: 0.4667
Epoch 3/1000
58/58 [==============================] - 19s 320ms/step - loss: 0.8648 - accuracy: 0.4569 - val_loss: 0.7380 - val_accuracy: 0.4667
Epoch 4/1000
58/58 [==============================] - 19s 328ms/step - loss: 0.8655 - accuracy: 0.4483 - val_loss: 0.7349 - val_accuracy: 0.4667
Epoch 5/1000
58/58 [==============================] - 18s 311ms/step - loss: 0.8412 - accuracy: 0.5086 - val_loss: 0.7422 - val_accuracy: 0.4667
Epoch 6/1000
58/58 [==============================] - 18s 301ms/step - loss: 0.8488 - accuracy: 0.4828 - val_loss: 0.7372 - val_accuracy: 0.4667
Epoch 7/1000
58/58 [==============================] - 18s 309ms/step - loss: 0.8379 - accuracy: 0.4569 - val_loss: 0.7295 - val_accuracy: 0.466

In [23]:
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation accuracy: {accuracy * 100:.2f}%")

15/15 [==============================] - 1s 56ms/step - loss: 0.7193 - accuracy: 0.5333
Validation accuracy: 53.33%
